In [ ]:
from google.cloud import bigquery

PROJECT_ID="my-673-project"
DATASET_ID="bus673_compustat"

bq_client = bigquery.Client(project=PROJECT_ID)

azquery = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.AZUR`;
"""
caquery = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.CAUR`;
"""
flquery = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.FLUR`;
"""
nvquery = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.NVUR`;
"""
orquery = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.ORUR`;
"""
txquery = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.TXUR`;
"""
waquery = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.WAUR`;
"""

In [19]:
import pandas as pd # type: ignore
import matplotlib.pyplot as plt # type: ignore
import plotly.express as px # type: ignore

azdf = bq_client.query(azquery).to_dataframe()
cadf = bq_client.query(caquery).to_dataframe()
fldf = bq_client.query(flquery).to_dataframe()
nvdf = bq_client.query(nvquery).to_dataframe()
ordf = bq_client.query(orquery).to_dataframe()
txdf = bq_client.query(txquery).to_dataframe()
wadf = bq_client.query(waquery).to_dataframe()
azdf['state'] = 'AZ'
azdf.rename(columns={'AZUR': 'value'}, inplace=True)
cadf['state'] = 'CA'
cadf.rename(columns={'CAUR': 'value'}, inplace=True)
fldf['state'] = 'FL'
fldf.rename(columns={'FLUR': 'value'}, inplace=True)
nvdf['state'] = 'NV'
nvdf.rename(columns={'NVUR': 'value'}, inplace=True)
ordf['state'] = 'OR'
ordf.rename(columns={'ORUR': 'value'}, inplace=True)
txdf['state'] = 'TX'
txdf.rename(columns={'TXUR': 'value'}, inplace=True)
wadf['state'] = 'WA'
wadf.rename(columns={'WAUR': 'value'}, inplace=True)

In [20]:
df = pd.concat([azdf, cadf, fldf, nvdf, ordf, txdf, wadf], ignore_index=True)
df.shape

(4191, 3)

In [22]:
df.head()

,observation_date,value,state
0,2025-10-01,NaN,AZ
1,2024-03-01,3.3,AZ
2,2007-07-01,3.4,AZ
3,2024-02-01,3.4,AZ
4,2024-04-01,3.4,AZ


In [28]:
import statsmodels.formula.api as smf
import numpy as np

treat_year = pd.to_datetime(20160101, format='%Y%m%d')

df.dropna(inplace=True)

df['treated'] = np.where(df['state'] == 'CA', 1, 0)
df['post'] = (df['observation_date'] >= treat_year).astype(int)
df['did'] = df['treated'] * df['post']

model = smf.ols('value ~ treated + post + did', data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  value   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.127
Method:                 Least Squares   F-statistic:                     203.1
Date:                Thu, 22 Jan 2026   Prob (F-statistic):          5.96e-123
Time:                        19:18:52   Log-Likelihood:                -8834.5
No. Observations:                4185   AIC:                         1.768e+04
Df Residuals:                    4181   BIC:                         1.770e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.6531      0.037    178.632      0.000       6.580       6.726
treated        0.8207      0.099      8.328      0.000       0.627       1.014
post          -1.7677      0.084    -21.071      0.000      -1.932      -1.603
did           -0.1061      0.222     -0.478      0.633      -0.541       0.329
==============================================================================
Omnibus:                     1975.960   Durbin-Watson:                   0.159
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23449.048
Skew:                           1.944   Prob(JB):                         0.00
Kurtosis:                      13.925   Cond. No.                         7.68
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
lowerwindow = pd.to_datetime(20150701, format='%Y%m%d')
upperwindow = pd.to_datetime(20160701, format='%Y%m%d')
window = df[(df['observation_date'] > lowerwindow) & (df['observation_date'] < upperwindow)]
window = window[window['observation_date'] != treat_year]
event_model = smf.ols('value ~ treated + post + did', data = window).fit()
event_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  value   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     2.438
Date:                Thu, 22 Jan 2026   Prob (F-statistic):             0.0722
Time:                        19:48:56   Log-Likelihood:                -54.698
No. Observations:                  70   AIC:                             117.4
Df Residuals:                      66   BIC:                             126.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.4667      0.099     55.003      0.000       5.268       5.665
treated        0.3733      0.263      1.420      0.160      -0.152       0.898
post          -0.2467      0.141     -1.755      0.084      -0.527       0.034
did           -0.0533      0.372     -0.143      0.886      -0.796       0.689
==============================================================================
Omnibus:                        1.239   Durbin-Watson:                   0.272
Prob(Omnibus):                  0.538   Jarque-Bera (JB):                1.251
Skew:                           0.301   Prob(JB):                        0.535
Kurtosis:                       2.741   Cond. No.                         7.66
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""